In [1]:
from profiling.funs import load, my_key, _save
import pandas as pd

In [21]:
def check(keywords):
    def find(x):
        return any(
            [word in x for word in keywords]
        )
    return find

key_ops = ['sequential', 'gradient_tape', 'categorical_crossentropy', 'SGD']
key_typs = ['Dataset']

def no_idle(df):
    return df[~df['Operation'].apply(check(['IDLE'])) ]

def refresh(df):
    df = no_idle(df)
    df = df.drop(columns=['Total self-time on Host (%)', 'Avg. time (us)', 'Total time (us)'])
    
    return df

def total_on(df, words):
    mask = df['Operation'].apply(check(words))

    return df[mask]['Avg. self-time (us)'].sum()

def from_set(fz):
    return {name:value for (name,value) in fz}

def extract(key, value):
    dct=from_set(key)
    num=total_on(value, dct['model_str'].upper())
    dct['avg_time_us'] = num
    
    return dct

In [40]:
df_dict = load('profiling/results/data_1.tflow')

In [30]:
for key in df_dict:
    break

In [37]:
df = df_dict[key]

In [38]:
df[df['Operation'].apply(check(['sequential']))]

,Type,Operation,#Occurrences,Avg. self-time (us),Total self-time (us),Total self-time on Host (%),Avg. time (us),Total time (us)
17,BiasAdd,sequential/DENSE/BiasAdd,1570,18542.356876,2.911150e+07,4.268746e-03,18542.356876,2.911150e+07
18,BiasAddGrad,gradient_tape/sequential/DENSE/BiasAdd/BiasAdd...,1570,18017.862368,2.828804e+07,4.147999e-03,18017.862368,2.828804e+07
19,BiasAddGrad,gradient_tape/sequential/FINAL_DENSE/BiasAdd/B...,1570,15.826468,2.484755e+04,3.643505e-06,15.826468,2.484755e+04
26,Cast,sequential/Cast,1570,1693.350358,2.658560e+06,3.898362e-04,1693.350358,2.658560e+06
28,ConcatV2,sequential/DENSE/Tensordot/concat,2,12.653000,2.530600e+01,3.710729e-09,12.653000,2.530600e+01
29,ConcatV2,sequential/DENSE/Tensordot/concat_1,1570,8.970846,1.408423e+04,2.065232e-06,8.970846,1.408423e+04
48,GatherV2,sequential/DENSE/Tensordot/GatherV2,1570,11.482906,1.802816e+04,2.643548e-06,11.482906,1.802816e+04
49,GatherV2,sequential/DENSE/Tensordot/GatherV2_1,1570,7.405814,1.162713e+04,1.704936e-06,7.405814,1.162713e+04
62,MatMul,gradient_tape/sequential/DENSE/Tensordot/MatMu...,1570,34831.254614,5.468507e+07,8.018710e-03,34831.254614,5.468507e+07
63,MatMul,gradient_tape/sequential/FINAL_DENSE/MatMul,1570,141224.933420,2.217231e+08,3.251223e-02,141224.933420,2.217231e+08


In [41]:
biglist=[extract(key, value) for key,value in df_dict.items()]

In [42]:
data=pd.DataFrame(data=biglist)

In [43]:
data

,avg_time_us,batch,hp,model_str,nodes,numf
0,1.421974e+06,32,128,dense,2,128
1,2.063369e+04,64,2,avg1d,2,64
2,2.604024e+04,512,2,conv1d,3,16
3,3.143445e+04,32,4,max2d,3,16
4,7.044036e+05,512,8,conv2d,3,64
5,4.418931e+04,64,4,max2d,1,32
6,8.745179e+05,256,2,max2d,3,128
7,7.577828e+03,128,2,avg2d,1,16
8,1.794819e+05,128,4,conv2d,1,64
9,3.565820e+05,256,4,conv2d,2,64


In [25]:
for name in ['conv1d', 'conv2d', 'avg1d', 'avg2d', 'max1d', 'max2d', 'dense', 'norm1d', 'norm2d']:
    _save((data[data['model_str']==name]).drop(columns='model_str'),'datasets/ops_total/{}.tflow'.format(name))

In [44]:
_save(data,'datasets/ops_total.tflow')